In [1]:
import pandas as pd
import pyecharts.options as opts
from pyecharts.charts import Map

In [2]:
from pyecharts.globals import CurrentConfig, NotebookType
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB

### 读取数据

In [22]:
df = pd.read_csv('二手房数据.csv', encoding = 'gbk')  # https://docs.python.org/3/library/codecs.html#standard-encodings
df.head()

,市区,小区,户型,朝向,楼层,装修情况,电梯,面积(㎡),价格(万元),年份
0,朝阳,育慧里一区,1室0厅,西,7,精装,有电梯,52.0,343.0,2001
1,朝阳,大西洋新城A区,2室2厅,南北,10,精装,有电梯,86.0,835.0,1999
2,朝阳,团结湖路,2室1厅,东西,6,精装,无电梯,65.0,430.0,1980
3,朝阳,尚家楼48号院,2室1厅,南北,12,精装,有电梯,75.0,610.0,1998
4,朝阳,望京西园一区,3室2厅,南北,6,精装,无电梯,115.0,710.0,1997


### 查看数据描述

In [23]:
df.describe()

,楼层,面积(㎡),价格(万元),年份
count,23677.000000,23677.000000,23677.000000,23677.000000
mean,12.764877,99.388616,610.668319,2001.326519
std,7.644138,51.270606,411.452107,9.001996
min,1.000000,15.000000,60.000000,1950.000000
25%,6.000000,66.000000,365.000000,1997.000000
50%,11.000000,88.000000,499.000000,2003.000000
75%,18.000000,118.000000,717.000000,2007.000000
max,57.000000,1019.000000,6000.000000,2017.000000


### 查看是否有缺失数据

In [5]:
df.isna().sum()

市区           0
小区           0
户型           0
朝向           0
楼层           0
装修情况         0
电梯        8257
面积(㎡)        0
价格(万元)       0
年份           0
dtype: int64

### 查看电梯列共有几种值

In [6]:
df['电梯'].unique()    # unique 给出Series唯一值

array(['有电梯', '无电梯', nan], dtype=object)

### 填充缺失值

In [7]:
df['电梯'].fillna('未知', inplace = True)

In [8]:
df.isna().sum()

市区        0
小区        0
户型        0
朝向        0
楼层        0
装修情况      0
电梯        0
面积(㎡)     0
价格(万元)    0
年份        0
dtype: int64

In [9]:
df['电梯'].unique()

array(['有电梯', '无电梯', '未知'], dtype=object)

### 查看房屋朝向数据

In [10]:
df['朝向'].unique()

array(['西', '南北', '东西', '南西', '西南', '东南', '南', '东北', '东', '西北', '北'],
      dtype=object)

朝向数据里面包含了“西南”和“南西”两个方向，合并为一个方向“西南”

In [11]:
df['朝向'] = df['朝向'].str.replace('南西','西南')
df['朝向'].unique()

array(['西', '南北', '东西', '西南', '东南', '南', '东北', '东', '西北', '北'],
      dtype=object)

### 统计各城区二手房数量

In [12]:
g = df.groupby('市区')
df_region = g.count()['小区']
region = df_region.index.tolist()
count = df_region.values.tolist()
df_region.sort_values(ascending = False)

市区
海淀     2983
朝阳     2973
丰台     2952
昌平     2811
西城     2130
大兴     2115
通州     1602
东城     1533
房山     1442
顺义     1221
石景山     882
门头沟     496
延庆      469
平谷       41
怀柔       15
密云       12
Name: 小区, dtype: int64

### 北京各城区二手房数量地图分布

In [24]:
new = [x + '区' for x in region]   # 源文件里没有'区',无法和Map()里数据对应
m = (
    Map()
    .add('',[list(z) for z in zip(new,count)],'北京')
    .set_global_opts(
        title_opts = opts.TitleOpts(title = '北京各城区二手房分布'),
        visualmap_opts = opts.VisualMapOpts(max_ = 3000)
    )
)

m.load_javascript()

In [25]:
m.render_notebook()

### 各城区二手房数量 -- 平均单价

In [26]:
df_price = g.mean()['价格(万元)']
df_price

市区
东城     851.425245
丰台     525.103591
大兴     460.469693
密云     425.333333
平谷     308.658537
延庆     549.876333
怀柔     785.200000
房山     360.611859
昌平     469.230345
朝阳     757.320148
海淀     827.740194
石景山    468.926757
西城     828.909202
通州     455.107553
门头沟    388.054032
顺义     558.339885
Name: 价格(万元), dtype: float64